In [2]:
import matplotlib.pyplot as pyplot
from gurobipy import *
import numpy as np
import os
import json

# Données

In [3]:
data_dir = '../data'
data = json.load(open(os.path.join(data_dir, 'large_instance.json')))

In [4]:
data

{'horizon': 36,
 'qualifications': ['E', 'I', 'J', 'A', 'C', 'H', 'G', 'D', 'B', 'F'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['B', 'A', 'C'],
   'vacations': [1, 2, 3, 4, 5]},
  {'name': 'Liam',
   'qualifications': ['E', 'A', 'D'],
   'vacations': [6, 7, 8, 9, 10]},
  {'name': 'Emma',
   'qualifications': ['B', 'H'],
   'vacations': [21, 22, 23, 24, 25]},
  {'name': 'Noah',
   'qualifications': ['C', 'H', 'G', 'D'],
   'vacations': [26, 27, 28, 29, 30]},
  {'name': 'Amelia',
   'qualifications': ['E', 'I', 'G', 'J', 'F'],
   'vacations': []},
  {'name': 'Oliver',
   'qualifications': ['J', 'F', 'G'],
   'vacations': [32, 33, 31]}],
 'jobs': [{'name': 'Job1',
   'gain': 15,
   'due_date': 26,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 4}},
  {'name': 'Job2',
   'gain': 30,
   'due_date': 22,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 2, 'C': 1, 'D': 4}},
  {'name': 'Job3',
   'gain': 30,
   'due_date': 16,
   'd

 # Modèle

In [5]:
m = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-28


# Utils

In [6]:
# quelques données utiles
n_staff = len(data['staff'])
n_jobs = len(data['jobs'])
horizon = data['horizon']
n_qualifications = len(data['qualifications'])

#relier les qualification (A, B, C, ...) à leur index dans les matrices de variables
qualification_to_idx = {value: idx for idx, value in enumerate(data['qualifications'])}

#données binaire pour savoir si un collaborateur a une qualification
has_qualification = np.zeros((n_staff, n_qualifications), dtype=np.int)
for staff_idx in range(n_staff):
    for qualification in data['qualifications']:
        has_qualification[staff_idx, qualification_to_idx[qualification]] = qualification in data['staff'][staff_idx]['qualifications']

<ipython-input-6-505af2d5906c>:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  has_qualification = np.zeros((n_staff, n_qualifications), dtype=np.int)


# Variables

In [7]:
### variables de décision ###

#vecteur binaire pour savoir si un projet est réalisé
job_is_done = m.addMVar(
    (n_jobs), vtype=GRB.BINARY
)

#matrice binaire pour savoir si un collaborateur c travail sur le projet p le jour j sur le qualification s
work = m.addMVar(
    (n_staff, 
     n_jobs, 
     horizon, 
     n_qualifications), vtype=GRB.BINARY
)

#matrice binaire pour savoir à quels projets à participé chaque colaborateur
participate = m.addMVar(
    (n_staff,
     n_jobs), vtype=GRB.BINARY
)

#matrice binaire pour savoir quels projets sont en cours quels jours
job_is_active = m.addMVar(
    (n_jobs,
     horizon), vtype=GRB.BINARY
)

### variables de modélisation ###

#vecteur pour connaître la date de réalisation de début de projet
start_date = m.addMVar(
    (n_jobs), vtype=GRB.INTEGER
)

#vecteur pour connaître la date de réalisation de chaque projet
finish_date = m.addMVar(
    (n_jobs), vtype=GRB.INTEGER
)

#vecteur binaire pour savoir si un job est temriné en retard
job_has_delay = m.addMVar(
    (n_jobs), vtype=GRB.BINARY
)

#vecteur des pénalités par projet
job_penalty = m.addMVar(
    (n_jobs), vtype=GRB.INTEGER
)

#vecteur des profits par propet
job_profit = m.addMVar(
    (n_jobs), vtype=GRB.INTEGER
)

#variable qui représente le nombre de jobs de l'employé qui en a le plus
max_job_per_staff = m.addVar(vtype=GRB.INTEGER)

#variable qui représente le nombre de jour que prend le projet le plus long
max_len_job = m.addVar(vtype=GRB.INTEGER)

m.update()

# Contraintes

In [8]:
M = 10000
epsilon = 0.01

In [9]:
# Contrainte de couverture des tâches d'un projet
for job_idx, job in enumerate(data['jobs']):
    for qualification, quantity in job['working_days_per_qualification'].items():
        if qualification in job['working_days_per_qualification'].keys():
            m.addConstr(work[:, job_idx, :, qualification_to_idx[qualification]].sum() <= job['working_days_per_qualification'][qualification])
        else:
            m.addConstr(work[:,job_idx,:,qualification_to_idx[qualification]].sum() <= 0)

# Contrainte de réalisation d'un projet selon la réalisation de toute les tâches de ce projet
for job_idx, job in enumerate(data['jobs']):
    for qualification, quantity in job['working_days_per_qualification'].items():
        m.addConstr(work[:, job_idx, :, qualification_to_idx[qualification]].sum() >= quantity - 1 + epsilon - M * (1 - job_is_done[job_idx])) #x = 0
        m.addConstr(work[:, job_idx, :, qualification_to_idx[qualification]].sum() <= quantity - 1 + M * job_is_done[job_idx]) #x = 1

# Contrainte d’unicite de l’affectation quotidienne du personnel (1 seule tâche sur 1 seul projet par jour)
for staff_idx, staff in enumerate(data['staff']):
    for day in range(horizon):
        m.addConstr(work[staff_idx, :, day, :].sum() <= 1)
        
# Contrainte de qualification du personnel (un collaborateur doit posséder la qualification sur laquelle il travaille)
for staff_idx, staff in enumerate(data['staff']):
    for qualification in data['qualifications']:
        if qualification not in staff['qualifications']:
            m.addConstr(work[staff_idx, :, :, qualification_to_idx[qualification]].sum() <= 0)
        
# Contrainte de vacances
for staff_idx, staff in enumerate(data['staff']):
    for vacation_day in staff['vacations']:
        m.addConstr(work[staff_idx, :, vacation_day - 1, :].sum() <= 0)
        
# Contrainte de participation d'un colaborateur à un projet (p = 1 si le colaborateur a travaillé au moins 1 jour sur une compétence sur ce projet)
for staff_idx in range(n_staff):
    for job_idx in range(n_jobs):
        m.addConstr(work[staff_idx, job_idx].sum() >= epsilon - M * (1 - participate[staff_idx, job_idx]) ) # force participate = 0 
        m.addConstr(work[staff_idx, job_idx].sum() <= M * participate[staff_idx, job_idx])  # force participate = 1 
        
# Contrainte de caractérisation de la variable job_is_active
for job_idx, job in enumerate(data['jobs']):
    for day in range(horizon):
        # Contrainte dans les deux sens car le comportement dépend de l'objectif fixé
        m.addConstr(work[:, job_idx, day, :].sum() >= epsilon - M * (1 - job_is_active[job_idx, day]))  # force job_is_active = 0
        m.addConstr(work[:, job_idx, day, :].sum() <= + M * job_is_active[job_idx, day])   # force job_is_active = 1

# Contrainte de caractérisation des variables start_date et finish_date
for job_idx, job in enumerate(data['jobs']):
    m.addConstr(start_date[job_idx] >= 0)
    m.addConstr(finish_date[job_idx] <= horizon - 1)
    for day in range(horizon):
        m.addConstr(start_date[job_idx] <= day * job_is_active[job_idx, day])   # en réalité ce n'est que le plus petit jour qui induit start_date
        m.addConstr(finish_date[job_idx] >= day * job_is_active[job_idx, day])  # en réalité ce n'est que le plus grand jour qui inudit finish_date
        
# Contrainte de caractérisation de la variable job_has_delay
for job_idx, job in enumerate(data['jobs']):
    # Je mes les deux sens de contrainte car selon le cas il y a un interêt à job_has_delay = 0 ou 1
    m.addConstr(finish_date[job_idx] >= job['due_date']-1 + epsilon - M * (1 - job_has_delay[job_idx]))
    m.addConstr(finish_date[job_idx] <= job['due_date']-1 + M * job_has_delay[job_idx])
    
# Contrainte de caractérisation de la variable job_penalty
for job_idx, job in enumerate(data['jobs']):
    m.addConstr(job_penalty[job_idx] == job_has_delay[job_idx] * job['daily_penalty'] * (finish_date[job_idx] - start_date[job_idx]))
    
# Contrainte de caractérisation de job_profit
for job_idx, job in enumerate(data['jobs']):
    m.addConstr(job_profit[job_idx] == job_is_done[job_idx] * (job['gain'] - job_penalty[job_idx]))

# Contrainte de définition de max_job_per_staff
for staff_idx in range(n_staff):
    m.addConstr(max_job_per_staff >= participate.sum())
    
# Contrainte de définition de max_len_job
for job_idx, job in enumerate(data['jobs']):
    m.addConstr(max_len_job >= finish_date[job_idx] - start_date[job_idx])

m.update()

# Résolution

## Large dataset

### Combination 1: priority on objective 1

In [11]:
# Objectifs 

# Objectif 1
m.setObjectiveN(-job_profit.sum(), 0, 1)

# Objectif 2
m.setObjectiveN(max_job_per_staff, 1, 0)

# Objectif 3
m.setObjectiveN(max_len_job, 2, 0)

m.params.outputflag = 0
m.optimize()

# Query number of multiple objectives, and number of solutions
nSolutions  = m.SolCount
nObjectives = m.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')
print(' ')

# For each solution, print value of first three variables, and
# value for each objective function
solutions = []
for s in range(nSolutions):
    # Set which solution we will query from now on
    m.params.SolutionNumber = s

    # Print objective value of this solution in each objective
    print('Solution', s, ':', end='')
    for o in range(nObjectives):
        # Set which objective we will query
        m.params.ObjNumber = o
        # Query the o-th objective value
        print(' ',m.ObjNVal, end='')
    
    # print variable vlaues for this solution
    print('')
    print('job_is_done = {}'.format(job_is_done.Xn))
    print('job_penalty = {}'.format(job_penalty.Xn))
    print('finish_date = {}'.format(finish_date.Xn))
    print('participate = {}'.format(participate.Xn))
    print('')

Problem has 3 objectives
Gurobi found 9 solutions
 
Solution 0 :  -800.0  35.0  34.0
job_is_done = [ 1. -0.  1.  1.  1.  1.  1.  1.  1. -0. -0.  1. -0.  0.  1.  1. -0. -0.
  0.  1.  1.  1.  1. -0.  1.]
job_penalty = [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. 102.   0.   0.   0.   0.   0.   0.   0.   0.]
finish_date = [25.  0. 15. 17. 13. 31. 17. 29. 19.  0.  0. 25.  0.  0. 34. 17. 34.  0.
  0. 11. 27. 25. 11.  0. 13.]
participate = [[ 1. -0.  1.  1. -0.  1.  0.  1.  0.  0.  0.  0. -0.  0.  0.  0. -0. -0.
  -0. -0. -0. -0.  0. -0. -0.]
 [ 0.  0.  0.  0.  1.  1. -0.  1.  0. -0. -0.  1.  0.  0.  1.  1.  0. -0.
  -0. -0. -0. -0. -0. -0. -0.]
 [-0.  0. -0. -0.  1.  1.  1. -0.  1. -0. -0. -0. -0. -0.  1.  0. -0. -0.
   0.  0. -0. -0.  1. -0. -0.]
 [-0.  0.  0. -0.  1.  0. -0.  1.  1.  0.  0.  0.  0.  0.  1.  0.  0. -0.
   0.  1.  1.  0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.  1.  1.  1.  0. -0.  0.  0. -0.  0.  1.  1.  0.  0.
   0.  0.  1. -0.  1.  0.  0.]


### Combination 2: every objective has same priority

In [10]:
# Objectifs 

# Objectif 1
m.setObjectiveN(-job_profit.sum(), 0, 0)

# Objectif 2
m.setObjectiveN(max_job_per_staff, 1, 0)

# Objectif 3
m.setObjectiveN(max_len_job, 2, 0)

m.params.outputflag = 0
m.optimize()

# Query number of multiple objectives, and number of solutions
nSolutions  = m.SolCount
nObjectives = m.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')
print(' ')

# For each solution, print value of first three variables, and
# value for each objective function
solutions = []
for s in range(nSolutions):
    # Set which solution we will query from now on
    m.params.SolutionNumber = s

    # Print objective value of this solution in each objective
    print('Solution', s, ':', end='')
    for o in range(nObjectives):
        # Set which objective we will query
        m.params.ObjNumber = o
        # Query the o-th objective value
        print(' ',m.ObjNVal, end='')
    
    # print variable vlaues for this solution
    print('')
    print('job_is_done = {}'.format(job_is_done.Xn))
    print('job_penalty = {}'.format(job_penalty.Xn))
    print('finish_date = {}'.format(finish_date.Xn))
    print('participate = {}'.format(participate.Xn))
    print('')

Problem has 3 objectives
Gurobi found 10 solutions
 
Solution 0 :  -800.0  35.0  34.0
job_is_done = [ 1. -0.  1.  1.  1.  1.  1.  1.  1. -0. -0.  1. -0. -0.  1.  1. -0. -0.
 -0.  1.  1.  1.  1. -0.  1.]
job_penalty = [ -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
   0.  -0.  -0. 102. 102.  -0.  -0.  -0.  -0.   0.  -0.]
finish_date = [25. 21. 15. 17. 13. 31. 17. 29. 19. -0. -0. 25. -0. -0. 34. 17. -0. 34.
 34. 11. 27. 25. 11.  0. 13.]
participate = [[ 1.  0.  1.  1. -0.  1. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0.  1.  1. -0.  1. -0. -0. -0.  1. -0. -0.  1.  1. -0. -0.
  -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0.  1.  1.  1. -0.  1. -0. -0. -0. -0. -0.  1. -0. -0. -0.
   0. -0. -0. -0.  1. -0. -0.]
 [-0. -0. -0. -0.  1. -0. -0. -0.  1. -0. -0. -0. -0. -0.  1. -0. -0. -0.
   0.  1.  1.  1. -0. -0. -0.]
 [-0. -0. -0. -0. -0.  1.  1.  1. -0. -0. -0. -0. -0. -0.  1.  1. -0. -0.
   0. -0.  1. -0.  1. -0. -0.]

## Medium instance